In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file_path = '../dataset/carbon_emission_dataset2.xlsx'
df=pd.read_excel(file_path)

In [ ]:
print("Dataset Loaded Successfully")
print("Shape",df.shape)



In [ ]:
df.sample(50)

In [ ]:
df.head(50)

In [ ]:
df.tail(50)

In [ ]:
df.describe()

In [ ]:
df.info()

CLEANING COLUMN NAMES

In [ ]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('/', '_')
)
print("Column names cleaned",df.columns)

CHECKING NULL VALUES

In [ ]:
print(df[['fuel_type', 'adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())

# Preview a few electric vehicle rows
df[df['fuel_type'].str.lower().str.contains('electric', na=False)][['adjusted_energy_kwh_per_km', 'total_energy_kwh']].head(10)


In [ ]:
#MAKING A COPY
df_cleaned = df.copy()

In [ ]:
print("\nEnergy column null counts in df_cleaned:")
print(df_cleaned[['adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())


In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.replace(['N/A', 'n/a', 'NA', '-', '--'], np.nan, inplace=True)
print("Placeholder values replaced with NaN",df.head(100))

In [ ]:
df_cleaned.info()

CHECKING DUPLICATES

In [ ]:
df_cleaned[df_cleaned.duplicated()]
df_cleaned.duplicated().sum()

In [ ]:
before = df_cleaned.shape[0]
df_cleaned.drop_duplicates(inplace=True)
after = df_cleaned.shape[0]
print(f"Dropped {before - after} duplicate rows")

In [ ]:
print("\n Missing values per column:\n")
print(df_cleaned.isnull().sum().sort_values(ascending=False))

In [ ]:
df_cleaned['logistics_partner'].value_counts()


COUNT VALUES

In [ ]:
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].str.strip().str.lower()
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].replace({
    'bluedart': 'bluedart',
    'ecom express': 'ecom express',
    'xpressbees': 'xpressbees',
    'shadowfax': 'shadowfax',
    'vrl logistics': 'vrl logistics',
    'tcl freight': 'tcl freight',
    'dtdc': 'dtdc',
    'loadshare': 'loadshare',
    'adani logistics': 'adani logistics',
    'concor': 'concor',
    'spicexpress': 'spicexpress',
    'gateway rai freight': 'gateway rail freight',  #typo
    'safeexpress': 'safeexpress',
    'quickjet airlines': 'quickjet airlines',
    'gati': 'gati',
    'amazon': 'amazon',
    'flipkart': 'flipkart',
    'delhivery': 'delhivery',
    'rivigo': 'rivigo',
})
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].str.title()
print(df_cleaned['logistics_partner'].value_counts())


In [ ]:
df_cleaned['vehicle_type'].value_counts()


In [ ]:
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].str.strip().str.lower()
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].replace({
    'van': 'van',
    'light truck': 'light_truck',
    'medium truck': 'medium_truck',
    'heavy truck': 'heavy_truck',
    'bike': 'bike',
    'auto': 'auto',
    'cargo train': 'train',
    'spice jet cargo': 'air_cargo',
    'cargo plane': 'air_cargo',
    'amazon air partner': 'air_cargo',
})
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].astype('category')
print("\nStandardized Vehicle Types:\n", df_cleaned['vehicle_type'].value_counts())


In [ ]:
df_cleaned['fuel_type'].unique()

In [ ]:
df_cleaned['fuel_type'] = df_cleaned['fuel_type'].astype(str).str.lower().str.strip()

df_cleaned['fuel_type'] = df_cleaned['fuel_type'].replace({
    'elecrtric': 'electric',
    'ev': 'electric',
})


In [ ]:
print(df_cleaned['fuel_type'].value_counts())


CLEANING NULL VALUES

In [ ]:
electric_mask = df_cleaned['fuel_type'].str.contains('electric', na=False)

# Nullify energy data for non-electric vehicles
df_cleaned.loc[~electric_mask, 'adjusted_energy_kwh_per_km'] = np.nan
df_cleaned.loc[~electric_mask, 'total_energy_kwh'] = np.nan

# Confirm
print(df_cleaned[['adjusted_energy_kwh_per_km', 'total_energy_kwh']].isnull().sum())


In [ ]:
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].str.lower().str.strip().str.replace('-', '', regex=False)
traffic_map = {
    'medium': 'moderate',
    'light': 'low',
    'high': 'heavy'
}

df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].replace(traffic_map)
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].str.capitalize()



In [ ]:
print(df_cleaned['traffic_condition'].value_counts())

In [ ]:
df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].str.upper().str.strip().str.replace('-', '', regex=False)

# Step 2: Create mapping dictionary
norm_map = {
    'BS3': 'BS-III',
    'BSIII': 'BS-III',
    'BS4': 'BS-IV',
    'BSIV': 'BS-IV',
    'BS6': 'BS-VI',
    'BSVI': 'BS-VI',
    'ELECTRIC': 'Electric',
    'TURBOPROP': 'Turboprop',
    'TURBOFAN': 'Turbofan',
    'JET': 'Jet',
    'NA': 'N/A',  # If NA means Electric
    'N/A': 'N/A',
    'NONE': 'N/A',
    np.nan: 'N/A'
}

df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].replace(norm_map)




In [ ]:
df_cleaned['engine_norm_type'] = df_cleaned['engine_norm_type'].fillna('Unknown')
df_cleaned['engine_norm_type'].sample(50)

In [ ]:
print("\nNormalized Engine Norm Types:\n", df_cleaned['engine_norm_type'].value_counts())

In [ ]:
df_cleaned.loc[~electric_mask, 'fuel_consumption_in_litre'] = (
    df_cleaned[~electric_mask]
    .groupby('vehicle_type')['fuel_consumption_in_litre']
    .transform(lambda x: x.fillna(x.mean()))
)


In [ ]:
df_cleaned['adjusted_fuel_efficiency_in_km_litre'].describe()


In [ ]:
df_cleaned.loc[electric_mask, 'adjusted_fuel_efficiency_in_km_litre'] = np.nan


In [ ]:
print(df_cleaned['adjusted_fuel_efficiency_in_km_litre'].isnull().sum())


MAPPING DATA TYPES OF FEATURES

In [ ]:
df_cleaned['no_of_stop'] = df_cleaned['no_of_stop'].astype(int)
df_cleaned['vehicle_age_in_years'] = df_cleaned['vehicle_age_in_years'].astype(int)
df_cleaned['traffic_condition'] = df_cleaned['traffic_condition'].astype('category')
df_cleaned['mode'] = df_cleaned['mode'].astype('category')
df_cleaned['fuel_type'] = df_cleaned['fuel_type'].astype('category')
df_cleaned['vehicle_type'] = df_cleaned['vehicle_type'].astype('category')
df_cleaned['logistics_partner'] = df_cleaned['logistics_partner'].astype('category')
df_cleaned['distance_in_km_per_route'] = df_cleaned['distance_in_km_per_route'].astype(int)

In [ ]:
df_cleaned[['distance_in_km_per_route', 'fuel_consumption_in_litre', 'adjusted_fuel_efficiency_in_km_litre', 'c02_emission_kg']].describe()


In [ ]:
print("\nFINAL MISSING VALUE REPORT:")
print(df_cleaned.isnull().sum().sort_values(ascending=False))


In [ ]:
df_cleaned.to_csv("../dataset/carbon_emission_cleaned.csv", index=False)